# Benchmarking cities in MENA

In support of an upcoming Urban flagship report, the MENA team is looking for a series of zonal statistics:

- Nighttime Lights, Population, and built-area:  
  - Entire FUA  
  - Its associated urban center / “core”  
  - Associated “periphery”  

The unit of analysis is the Functional Urban Areas (FUAs) from the [UCDB Database](https://human-settlement.emergency.copernicus.eu/ghs_stat_ucdb2015mt_r2019a.php). For each FUA, we need to grab the associated urban periphary (lower threshold urban areas)

In [22]:
import sys
import os

import geopandas as gpd


sys.path.append("C:/WBG/Work/Code/GOSTrocks/src")

In [83]:
data_folder = "C:/WBG/Work/data"
ucdb_file = os.path.join(
    data_folder,
    "URBAN",
    "GHS_STAT_UCDB2015MT_GLOBE_R2019A",
    "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg",
)
fua_file = os.path.join(
    data_folder, "URBAN", "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
)

out_folder = "C:/WBG/Work/MENA_Urban/"
urban_res_folder = os.path.join(out_folder, "urban_data")

for out_folder in [urban_res_folder]:
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

urban_periphary_file = os.path.join(urban_res_folder, "urban_periphary.gpkg")
urban_core_file = os.path.join(urban_res_folder, "urban_core.gpkg")

In [108]:
inD = gpd.read_file(ucdb_file)
inF = gpd.read_file(fua_file)
inD = inD.to_crs(inF.crs)

In [85]:
inF.head()

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry
0,1.0,1.0,5959,Herat,1.0,AFG,Afghanistan,139.0,60.0,888811.175807,844574.564331,44236.611476,"MULTIPOLYGON (((5529000.000 4155000.000, 55310..."
1,192.0,1.0,5964,Guzarah,1.0,AFG,Afghanistan,32.0,16.0,169489.573231,160204.260864,9285.312366,"MULTIPOLYGON (((5541000.000 4139000.000, 55420..."
2,354.0,1.0,5968,Shindand,0.0,AFG,Afghanistan,12.0,12.0,147553.403870,147553.403870,0.000000,"MULTIPOLYGON (((5573000.000 4030000.000, 55740..."
3,505.0,1.0,5970,Qala i Naw,0.0,AFG,Afghanistan,3.0,3.0,79809.722656,79809.722656,0.000000,"MULTIPOLYGON (((5592000.000 4221000.000, 55900..."
4,648.0,1.0,5973,Farah,1.0,AFG,Afghanistan,32.0,16.0,131508.797060,122843.460327,8665.336733,"MULTIPOLYGON (((5607000.000 3923000.000, 56080..."


In [113]:
fua_peripheries = inF.copy()
for idx, row in inF.iterrows():
    # grab the related UCDBs
    ucdb_ids = row["UC_IDs"].split(";")
    ucdb_ids = [int(x) for x in ucdb_ids]
    sel_cores = inD.loc[inD["ID_HDC_G0"].isin(ucdb_ids)]
    periphery_geom = row["geometry"].difference(sel_cores.unary_union)
    fua_peripheries.loc[idx, "geometry"] = periphery_geom

fua_peripheries.to_file(os.path.join(out_folder, "FUA_peripheries.gpkg"), driver="GPKG")